# Investigate the Quality of Flower102

In [3]:
!pip install torchvision scienceplots einops torchinfo randomname seaborn -q


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.9 -m pip install --upgrade pip


In [4]:
from torchvision import datasets

# TODO: replace with package after public on PyPi
import sys

sys.path.append("../")

from src.cleaner.selfclean import PretrainingType, SelfClean

/home/fabian/Documents/_git/GitHub/digital-dermatology/SelfClean/examples/../ssl_library/src/utils/metrics.py:13: SyntaxWarning: "is" with a literal. Did you mean "=="?
  output=mask.int() if mode is "binary" else mask.argmax(dim=1).int(),
/home/fabian/Documents/_git/GitHub/digital-dermatology/SelfClean/examples/../ssl_library/src/utils/metrics.py:17: SyntaxWarning: "is" with a literal. Did you mean "=="?
  threshold=0.5 if mode is "binary" else None,


In [5]:
dataset = datasets.Flowers102(root='./tmp', download=True)

  0%|          | 0/344862509 [00:00<?, ?it/s]

Extracting tmp/flowers-102/102flowers.tgz to tmp/flowers-102


  0%|          | 0/502 [00:00<?, ?it/s]

  0%|          | 0/14989 [00:00<?, ?it/s]

In [6]:
selfclean = SelfClean()
out_dict = selfclean.run_on_dataset(
    dataset=dataset,
    pretraining_type=PretrainingType.DINO,
    num_workers=10,
)

Will run the code on one GPU.
STARTUP | distributed init (rank 0): env://


/home/fabian/.local/lib/python3.9/site-packages/torchvision/transforms/transforms.py:899: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


Running on: cuda
Run directory of model: /home/fabian/Documents/_git/GitHub/digital-dermatology/SelfClean/examples/DINO-Flowers102-wood-meat
Data loaded: there are 1020 train images, 32 batches, and a batch size of 4.
Single GPU detected, model will run on single instance.
Single GPU detected, model will run on single instance.
Student and Teacher are built: they are both vit_tiny network.
Pre-trained weights not found. Training from scratch.


Epoch: 1, Train loss: 8.328475, Train stud/teach acc: 0.1786: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:26<00:00,  1.22it/s]
Epoch: 7, Train loss: 8.181155, Train stud/teach acc: 0.0000: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:20<00:00,  1.55it/s]
Epoch: 8, Train loss: 8.120199, Train stud/teach acc: 0.1071: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:20<00:00,  1.54it/s]
Epoch: 9, Train loss: 8.061924, Train stud/teach acc: 0.1786: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:20<00:00,  1.53it/s]
Epoch: 10, Train loss: 8.025546, Train stud/teach acc: 0.3571: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 

Epoch: 45, Train loss: 7.691669, Train stud/teach acc: 0.0000: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:21<00:00,  1.52it/s]
Epoch: 46, Train loss: 7.871750, Train stud/teach acc: 0.1786: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:20<00:00,  1.55it/s]
Epoch: 47, Train loss: 7.672338, Train stud/teach acc: 0.1429: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:20<00:00,  1.56it/s]
Epoch: 48, Train loss: 7.748367, Train stud/teach acc: 0.1786: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:20<00:00,  1.53it/s]
Epoch: 49, Train loss: 7.826695, Train stud/teach acc: 0.1071: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 

Saving checkpoint: /home/fabian/Documents/_git/GitHub/digital-dermatology/SelfClean/examples/DINO-Flowers102-wood-meat/checkpoints/checkpoint-epoch50.pth ...
Saving current best: model_best.pth ...


Processing possible near duplicates: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 52/52 [00:00<00:00, 102.61it/s]
Processing possible irrelevant samples: 2039it [00:00, 39924.12it/s]
/home/fabian/Documents/_git/GitHub/digital-dermatology/SelfClean/examples/../src/cleaner/label_errors/intra_extra_distance_mixin.py:30: RuntimeWarning: invalid value encountered in multiply
  min_same = np.nanmin((o_hot_same * self.distance_matrix), axis=-1)
/home/fabian/Documents/_git/GitHub/digital-dermatology/SelfClean/examples/../src/cleaner/label_errors/intra_extra_distance_mixin.py:31: RuntimeWarning: invalid value encountered in multiply
  min_diff = np.nanmin((o_hot_diff * self.distance_matrix), axis=-1)


In [8]:
out_dict

{'irrelevants': {'indices': array([207, 104, 449, ..., 898, 537, 628]),
  'scores': array([0.92241525, 0.92565718, 0.94551118, ..., 0.9879522 , 0.9879522 ,
         0.9879522 ])},
 'near_duplicates': {'indices': array([[ 41, 474],
         [456, 890],
         [253, 435],
         ...,
         [144, 753],
         [404, 753],
         [156, 753]], dtype=int32),
  'scores': array([4.6694448e-04, 4.8342266e-04, 7.6659251e-04, ..., 8.3393097e-01,
         8.3463347e-01, 8.5160631e-01], dtype=float32)},
 'label_errors': {'indices': array([331, 421, 942, ..., 206, 546, 540]),
  'scores': array([2.92874383e-04, 4.69011206e-04, 4.83882883e-04, ...,
         8.97479150e-01, 9.27146356e-01, 9.41742379e-01])}}